# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

### Mongoimport Command

The following `mongoimport` command is used to import the `establishments.json` file into MongoDB.

- The database is named `uk_food`.
- The collection is named `establishments`.
- The command also drops any existing `establishments` collection to ensure we are working with fresh data.

```bash
mongoimport --db uk_food --collection establishments --drop --file establishments.json

> **Note**: The `mongoimport` command was executed in the command line before running this notebook. This notebook assumes that the data is already imported into MongoDB.


In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())


['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
print(db.list_collection_names())


['establishments']


In [6]:
# review a document in the establishments collection
establishments = db['establishments']
pprint(establishments.find_one())


{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f4814bece627a9963f079e'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",  # We'll update this later
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",  # No phone number provided
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",  # No hygiene score yet
        "Structural": "",  # No structural score yet
        "ConfidenceInManagement": ""  # No confidence score yet
    },
    "SchemeType": "FHRS",  # Food Hygiene Rating Scheme
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",  # Not filled yet
    "Distance": 4623.9723280747176,  # Distance from reference location
    "NewRatingPending": True  # Rating is pending for this new restaurant
}


In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

InsertOneResult(ObjectId('66f48180156f91bb3ec72d7b'), acknowledged=True)

In [10]:
# Check that the new restaurant was inserted
result = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(result)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f48180156f91bb3ec72d7b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)
pprint(result)


{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(
    {"BusinessName": "Penang Flavours"},  # Filter to find the correct restaurant
    {"$set": {"BusinessTypeID": result['BusinessTypeID']}}  # Update the BusinessTypeID
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [13]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(updated_restaurant)


{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f48180156f91bb3ec72d7b'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {dover_count}")


Number of documents with LocalAuthorityName as 'Dover': 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print(f"Number of documents deleted: {delete_result.deleted_count}")


Number of documents deleted: 994


In [16]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover' after deletion: {dover_count_after}")


Number of documents with LocalAuthorityName as 'Dover' after deletion: 0


In [17]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one({"LocalAuthorityName": {"$ne": "Dover"}})
pprint(remaining_document)


{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66f4814bece627a9963f0a85'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
# Convert longitude to decimal
establishments.update_many(
    {},
    [{"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}]
)

# Convert latitude to decimal
establishments.update_many(
    {},
    [{"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}}]
)



UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [19]:
updated_document = establishments.find_one()
pprint(updated_document['geocode'])


{'latitude': 51.083812, 'longitude': 1.195625}


Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Change the data type from String to Integer for RatingValue

establishments.update_many(
    {"RatingValue": {"$ne": None}},  # Ensure RatingValue is not null
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}]
)


UpdateResult({'n': 34694, 'nModified': 34694, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
# Check that the coordinates and rating value are now numbers
# Find one document where RatingValue is not null to check both RatingValue and geocode (longitude/latitude)
document_check = establishments.find_one({"RatingValue": {"$ne": None}})
pprint({
    "longitude": document_check['geocode']['longitude'],
    "latitude": document_check['geocode']['latitude'],
    "RatingValue": document_check['RatingValue']
})


{'RatingValue': 5, 'latitude': 51.083812, 'longitude': 1.195625}


In [23]:
# Find one document where RatingValue is not null
document_check = establishments.find_one({"RatingValue": {"$ne": None}})

# Print the types of longitude, latitude, and RatingValue
longitude_type = type(document_check['geocode']['longitude'])
latitude_type = type(document_check['geocode']['latitude'])
rating_value_type = type(document_check['RatingValue'])

print(f"Longitude type: {longitude_type}")
print(f"Latitude type: {latitude_type}")
print(f"RatingValue type: {rating_value_type}")


Longitude type: <class 'float'>
Latitude type: <class 'float'>
RatingValue type: <class 'int'>
